In [4]:

!pip install -qU langgraph langchain-core langchain langchain-openai langchain-community google-api-python-client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.4/438.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 12.9 MB/s eta 0:00:00


In [6]:
!pip install langchain-google-genai

# --- REQUIRED IMPORTS ---
import os
import re
from typing import List, Dict, Any, TypedDict

# Langchain core components
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Langgraph specific
from langgraph.graph import StateGraph, END

# Real LLM integration (using Google Generative AI as an example)
from langchain_google_genai import ChatGoogleGenerativeAI # <<< CHANGED THIS LINE

# Real Search Tool integration (using Google Custom Search)
from langchain_community.tools import GoogleSearchRun
from langchain_community.utilities import GoogleSearchAPIWrapper

# Agent Executor for ToolAgent
from langchain.agents import AgentExecutor, create_tool_calling_agent
try:
    # For Google Colab Secrets
    from google.colab import userdata
    # os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') # <<< REMOVED/COMMENTED OUT THIS LINE
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')
    print("API keys loaded from Google Colab secrets.")
except ImportError:
    # For local environment, ensure these are set as environment variables
    print("Not in Colab. Ensure GOOGLE_API_KEY and GOOGLE_CSE_ID are set as environment variables.")
    if not os.getenv("GOOGLE_API_KEY") or not os.getenv("GOOGLE_CSE_ID"):
        raise ValueError("API keys for Google API or Google CSE ID are not set. "
                         "Please set them as environment variables or Colab secrets.")


# --- 1. DEFINE THE GRAPH STATE ---
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        user_query (str): The original user query.
        sub_tasks (List[Dict[str, Any]]): A list of sub-tasks generated by the PlanAgent.
                                          Each sub-task can have fields like 'description', 'status', 'result'.
        current_task_index (int): The index of the sub-task currently being processed.
        tools (List[str]): List of tool names available to the ToolAgent.
        reflection (str): Feedback or reflection from the Inner Loop to the PlanAgent.
        final_result (str): The accumulated final result of all sub-tasks.
        iteration_count (int): To keep track of the refinement iterations.
    """
    user_query: str
    sub_tasks: List[Dict[str, Any]]
    current_task_index: int
    tools: List[str]
    reflection: str
    final_result: str
    iteration_count: int


# --- 2. INITIALIZE REAL LANGUAGE MODELS AND TOOLS ---

# Initialize a real LLM (using Google's Gemini-Pro) <<< CHANGED THIS LINE
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0) # <<< CHANGED THIS LINE

# Define real tools
# Google Search Tool
search_wrapper = GoogleSearchAPIWrapper()
Google_Search_tool = GoogleSearchRun(api_wrapper=search_wrapper)

# Simple Calculator Tool (be cautious with eval() in production for security)
@tool
def calculator_tool(expression: str) -> str:
    """Evaluates a mathematical expression."""
    print(f"Executing calculation for: {expression}")
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error evaluating expression: {e}"

# List of actual tools available to the ToolAgent
tools = [Google_Search_tool, calculator_tool]
# The names in the tools list here are the actual tool function objects,
# which the AgentExecutor will use.


# --- 3. IMPLEMENT THE PLANAGENT NODE ---
def plan_agent_node(state: GraphState) -> GraphState:
    """
    PlanAgent: Splits the user query into sub-tasks or refines existing sub-tasks based on reflection.
    Uses a real LLM for planning.
    """
    user_query = state["user_query"]
    sub_tasks = state["sub_tasks"]
    reflection = state["reflection"]
    iteration_count = state["iteration_count"]

    print(f"\n--- PlanAgent (Iteration {iteration_count}) ---")
    print(f"User Query: {user_query}")
    print(f"Existing Sub-tasks: {sub_tasks}")
    print(f"Reflection from previous step: {reflection}")

    prompt_messages = [
        ("system", """You are a PlanAgent. Your goal is to break down complex user queries into smaller, manageable sub-tasks.
         Each sub-task should be clear, actionable, and ideally solvable by a specialized tool (like search or calculator).
         If 'reflection' is provided, refine the existing sub-tasks (modify, delete, or add new ones) based on the feedback.
         Always output a JSON array of sub-tasks. Each sub-task *must* have a 'description' and a unique 'id' (e.g., "task_1").

         If refining (based on reflection), you can also include an 'action' key:
         - {{"id": "task_X", "action": "delete"}} to remove a task.
         - {{"id": "task_Y", "description": "New task", "action": "add"}} to add a new task.
         - {{"id": "task_Z", "description": "Modified task"}} to update an existing task.

         If no sub-tasks are needed (e.g., a simple direct question that can be answered immediately), return an empty array [].
         """),
        ("user", "User Query: {user_query}\nExisting Sub-tasks: {sub_tasks}\nReflection: {reflection}\nIteration Count: {iteration_count}")
    ]

    prompt_template = ChatPromptTemplate.from_messages(prompt_messages)
    parser = JsonOutputParser()
    chain = prompt_template | llm | parser

    try:
        response = chain.invoke({"user_query": user_query, "sub_tasks": sub_tasks, "reflection": reflection, "iteration_count": iteration_count})
        if not isinstance(response, list):
            print(f"Warning: PlanAgent response not a list: {response}. Attempting to parse.")
            response = [] # Fallback
        new_tasks_from_plan = response
    except Exception as e:
        print(f"Error invoking PlanAgent LLM: {e}. Falling back to default plan.")
        new_tasks_from_plan = [{"id": "task_1", "description": f"Address the query: {user_query}"}]


    # Apply refinements (add/delete/modify) from the LLM's response
    updated_sub_tasks_dict = {task.get('id'): task for task in sub_tasks if task.get('action') != 'delete'}

    task_id_counter = 0
    if updated_sub_tasks_dict:
        # Find max existing id to ensure new unique IDs
        max_id_num = 0
        for task_id in updated_sub_tasks_dict.keys():
            if task_id.startswith("task_") and task_id[5:].isdigit():
                max_id_num = max(max_id_num, int(task_id[5:]))
        task_id_counter = max_id_num

    final_sub_tasks_list = []
    for task_def in new_tasks_from_plan:
        if task_def.get("action") == "delete":
            if task_def.get("id") in updated_sub_tasks_dict:
                del updated_sub_tasks_dict[task_def.get("id")]
            continue
        elif task_def.get("action") == "add":
            task_id_counter += 1
            new_id = f"task_{task_id_counter}"
            task_def["id"] = new_id
            final_sub_tasks_list.append({k: v for k, v in task_def.items() if k != "action"})
        else: # Modify or new task without explicit "add"
            # Ensure unique IDs for newly generated tasks if LLM doesn't provide them unique
            if "id" not in task_def or task_def["id"] not in updated_sub_tasks_dict:
                 task_id_counter += 1
                 task_def["id"] = f"task_{task_id_counter}"

            final_sub_tasks_list.append({k: v for k, v in task_def.items() if k != "action"})

    # Ensure all tasks from the original set that weren't deleted or explicitly modified are carried forward
    # This might need more sophisticated merging if LLM returns a full new plan vs. incremental changes
    # For simplicity, we'll take the LLM's latest full plan if it's not empty, otherwise merge.
    if new_tasks_from_plan:
        new_sub_tasks = final_sub_tasks_list # Trust the LLM's latest full plan
    else:
        new_sub_tasks = list(updated_sub_tasks_dict.values()) # If LLM returned empty, use existing tasks (if not deleted)


    state["sub_tasks"] = new_sub_tasks
    state["current_task_index"] = 0  # Reset to the first task if planning/refinement occurs
    state["reflection"] = ""  # Clear reflection after it's used
    state["iteration_count"] += 1
    print(f"PlanAgent generated/updated sub-tasks: {state['sub_tasks']}")
    return state


#  IMPLEMENT THE TOOLAGENT NODE
def tool_agent_node(state: GraphState) -> GraphState:
    """
    ToolAgent: Executes the current sub-task using available tools.
    Uses a real LLM with AgentExecutor for tool selection and execution.
    """
    current_task_index = state["current_task_index"]
    sub_tasks = state["sub_tasks"]

    if current_task_index >= len(sub_tasks):
        print("ToolAgent: No more tasks to execute.")
        return state

    current_task = sub_tasks[current_task_index]
    task_description = current_task["description"]

    print(f"\n--- ToolAgent ---")
    print(f"Executing task ({current_task_index + 1}/{len(sub_tasks)}): {task_description}")

    # Define the tool agent prompt
    agent_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant. Use the provided tools to execute the given task. "
                   "If the task requires a search, use the 'Google Search_tool'. "
                   "If it requires a calculation, use the 'calculator_tool'. "
                   "If you have the final answer, output it directly. Do not make up information."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}") # This is where the agent's thoughts and tool calls go
    ])

    # Create the tool-calling agent
    agent = create_tool_calling_agent(llm, tools, agent_prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

    task_result = ""
    try:
        # Invoke the agent executor with the current task description
        response = agent_executor.invoke({"input": task_description})
        task_result = response.get("output", "No direct output from agent executor, check verbose logs.")
        if task_result.strip() == "":
            task_result = f"Agent executed task '{task_description}' but returned no explicit output. Check logs."

    except Exception as e:
        task_result = f"Error during tool execution for task '{task_description}': {e}"
        print(f"Error in ToolAgent execution: {e}")

    state["sub_tasks"][current_task_index]["result"] = task_result
    state["sub_tasks"][current_task_index]["status"] = "completed"
    state["current_task_index"] += 1
    state["final_result"] += f"\n- {task_description}: {task_result}"
    print(f"Task result: {task_result}")
    return state


# IMPLEMENT REFLECTION NODE
def reflection_node(state: GraphState) -> GraphState:
    """
    Reflection: Evaluates the outcome of the ToolAgent's execution and generates feedback for PlanAgent.
    Uses a real LLM for reflection.
    """
    current_task_index = state["current_task_index"]
    sub_tasks = state["sub_tasks"]
    user_query = state["user_query"]

    print(f"\n--- Reflection Node ---")

    # Determine if all planned tasks have been processed by ToolAgent
    if current_task_index >= len(sub_tasks):
        state["reflection"] = "All planned tasks processed. Ready for final review."
        print("Reflection: All planned tasks processed.")
        return state

    # Otherwise, reflect on the last completed task
    previous_task_index = current_task_index - 1
    reflection_feedback = "No specific feedback." # Default
    if previous_task_index >= 0:
        previous_task = sub_tasks[previous_task_index]
        task_result = previous_task.get("result", "")
        task_description = previous_task.get("description", "unknown task")
        task_status = previous_task.get("status", "unknown")

        reflection_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a Reflection Agent. Your job is to analyze the outcome of a completed task and provide feedback.
             Determine if the task was completed successfully, if it failed, or if it needs more information/refinement.
             If it failed or needs refinement, provide specific instructions for the PlanAgent to modify, delete, or add tasks.
             If successful, simply state that.
             Example feedback for refinement: "Task 'Research X' failed. The result was empty. Please refine the task to 'Search for X with alternative keywords'."
             Example for success: "Task 'Calculate Y' completed successfully."
             """),
            ("user", f"Original Query: {user_query}\nTask Description: {task_description}\nTask Result: {task_result}\nTask Status: {task_status}")
        ])

        chain = reflection_prompt | llm
        try:
            reflection_feedback = chain.invoke({"user_query": user_query, "task_description": task_description, "task_result": task_result, "task_status": task_status}).content
        except Exception as e:
            reflection_feedback = f"Error during reflection: {e}. Assuming task '{task_description}' needs review."

        print(f"Reflection for '{task_description}': {reflection_feedback}")
    else:
        print("Reflection: Initial reflection, no tasks processed yet.")

    state["reflection"] = reflection_feedback
    return state


# IMPLEMENT AGENT DISPATCH (CONDITIONAL EDGE LOGIC)

def should_continue_planning(state: GraphState) -> str:
    """
    Decides whether to continue with planning/refinement (Outer Loop)
    or move to task execution (Inner Loop), or finish.
    This is called FROM the 'reflection' node.
    """
    sub_tasks = state["sub_tasks"]
    current_task_index = state["current_task_index"]
    reflection = state["reflection"]

    print(f"\n--- Deciding Next Step from Reflection ---")
    print(f"Current reflection: '{reflection}'")
    print(f"Current task index: {current_task_index}, Total tasks: {len(sub_tasks)}")

    # If reflection indicates a need for refinement (e.g., a task failed)
    if "refine the plan" in reflection.lower() or "failed" in reflection.lower() or "needs more information" in reflection.lower():
        print("Decision: Go back to PlanAgent for replanning/refinement.")
        return "replan"

    # If all current tasks in the list have been processed by ToolAgent and the reflection confirms this, then the overall workflow is done.

    if current_task_index >= len(sub_tasks) and "all planned tasks processed" in reflection.lower():
        print("Decision: All tasks completed. END workflow.")
        return "end"

    # If there are still tasks in the current plan that haven't been processed yet
    if current_task_index < len(sub_tasks):
        print("Decision: Continue to ToolAgent for next task execution.")
        return "continue"

    # Fallback: If somehow the plan is empty or in an unexpected state after reflection
    print("Decision: Defaulting to replan (unexpected state).")
    return "replan"

def should_continue_task_execution(state: GraphState) -> str:
    """
    Decides whether to immediately execute tasks or go to reflection.
    This is called FROM the 'plan_agent' node.
    """
    sub_tasks = state["sub_tasks"]
    current_task_index = state["current_task_index"]

    print(f"\n Deciding Next Step from PlanAgent")
    print(f"Tasks planned: {len(sub_tasks)}, Current task index: {current_task_index}")

    # If the PlanAgent has generated tasks and there are tasks to execute
    if sub_tasks and current_task_index < len(sub_tasks):
        print("Decision: Tasks available. Continue to ToolAgent for execution.")
        return "continue" # More tasks to execute
    else:
        # If the plan is empty ( LLM decided no tasks needed, or initial plan is empty)
        print("Decision: No tasks from PlanAgent or plan is empty. Go to Reflection.")
        return "reflect" # Go to reflection_node  to handle empty plan or final review

#  BUILD THE LANGGRAPH GRAPH

# Create the graph
workflow = StateGraph(GraphState)

# Add nodes for each agent/component
workflow.add_node("plan_agent", plan_agent_node)
workflow.add_node("tool_agent", tool_agent_node)
workflow.add_node("reflection_node", reflection_node)

# Set the entry point for the graph
workflow.set_entry_point("plan_agent")
# From 'plan_agent': Decide whether to execute tasks or reflect (if plan is empty)
workflow.add_conditional_edges(
    "plan_agent",
    should_continue_task_execution,
    {
        "continue": "tool_agent",  # If tasks are ready, go to tool agent
        "reflect": "reflection_node"    # If no tasks from plan, go to reflection
    }
)

# From 'tool_agent': Always go to reflection after executing a task
workflow.add_edge("tool_agent", "reflection_node")

# From 'reflection_node': Decide whether to replan, continue tasks, or end the workflow
workflow.add_conditional_edges(
    "reflection_node",
    should_continue_planning,
    {
        "replan": "plan_agent", # Go back to PlanAgent for refinement
        "continue": "tool_agent", # Continue with next task in Inner Loop (if reflection allows)
        "end": END              # it will  terminate the workflow once all is doen
    }
)

# Compile the graph
app = workflow.compile()

def run_agent_workflow_from_user_input():
    """
    Function to run the agentic workflow by taking continuous user input.
    """
    print("--- Agentic Workflow powered by Langgraph (Real LLM & Tools) ---")
    print("Enter your queries below. Type 'exit' to quit.")

    while True:
        user_input_query = input("\nEnter your query: ").strip()
        if user_input_query.lower() == 'exit':
            print("Exiting workflow. Goodbye!")
            break

        if not user_input_query:
            print("Query cannot be empty. Please enter a query.")
            continue

        print(f"\n[SYSTEM] Processing query: '{user_input_query}'")

        # Initialize the state with the user's query for each new interaction
        initial_state = GraphState(
            user_query=user_input_query,
            sub_tasks=[],
            current_task_index=0,
            tools=["Google Search_tool", "calculator_tool"], # Names of the tools for conceptual tracking
            reflection="",
            final_result="",
            iteration_count=0
        )

        try:

            for s in app.stream(initial_state):
                pass # Prints are handled within the nodes

            # Get the final state after execution (the last state from the stream)
            final_state = app.invoke(initial_state)
            print("\n" + "="*25 + " FINAL RESULT " + "="*25)
            print("Workflow execution completed.")
            if final_state["final_result"]:
                print("Result Summary:")
                print(final_state["final_result"])
            else:
                print("No specific result generated for this query. The agent may have determined no action was needed or encountered an issue.")
            print("="*64 + "\n")

        except Exception as e:
            print(f"\n[ERROR] An unexpected error occurred during workflow execution: {e}")
            print("Please ensure your API keys are correctly configured and try another query.")


run_agent_workflow_from_user_input()

API keys loaded from Google Colab secrets.
--- Agentic Workflow powered by Langgraph (Real LLM & Tools) ---
Enter your queries below. Type 'exit' to quit.

Enter your query: what is machine learning


  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/


[SYSTEM] Processing query: 'what is machine learning'

--- PlanAgent (Iteration 0) ---
User Query: what is machine learning
Existing Sub-tasks: []
Reflection from previous step: 


  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/

Error invoking PlanAgent LLM: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/

Error in ToolAgent execution: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/

Error invoking PlanAgent LLM: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_